<a href="https://colab.research.google.com/github/ASBJ021/newproject/blob/master/clean_dirty_car_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the model architecture
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(64, 64, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(2, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Load and preprocess the dataset
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    "/content/folder/train",
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)
validation_generator = train_datagen.flow_from_directory(
    "/content/folder/valid",
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Save the trained model
model.save("/content/model")




In [1]:
!cp -r /content/drive/MyDrive/car-classification/folder   /content

In [14]:
# Load the saved model
loaded_model = tf.keras.models.load_model("/content/model")

# Perform real-time inference using the loaded model
# Load and preprocess the input image
image = tf.keras.preprocessing.image.load_img("/content/folder/test/Dirty/8170ffcc-eda4-4981-87f7-8a6dd5877761_jpg.rf.4d6fb379a0f397bb9fe8d90e46afb8ca.jpg", target_size=(64, 64))
input_image = tf.keras.preprocessing.image.img_to_array(image)
input_image = input_image / 255.0
input_image = tf.expand_dims(input_image, 0)

# Perform inference
predictions = loaded_model.predict(input_image)
print('predictions: ', predictions)
class_index = tf.argmax(predictions[0])
class_label = class_index.numpy()

# Print the predicted class label
print("Predicted class label:", class_label)

1/1 [==============================] - 0s 105ms/step
predictions:  [[0.29004508 0.70995486]]
Predicted class label: 1


In [ ]:
import os
import csv
import tensorflow as tf

# Load the saved model
loaded_model = tf.keras.models.load_model("/content/folder/model")

# Folder path containing the images
folder_path = "/content/folder/test/Clean"

# Create a list to store the results
results = []

# Iterate over the images in the folder
for filename in os.listdir(folder_path):
    # Load and preprocess the input image
    image_path = os.path.join(folder_path, filename)
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
    input_image = tf.keras.preprocessing.image.img_to_array(image)
    input_image = input_image / 255.0
    input_image = tf.expand_dims(input_image, 0)

    # Perform inference
    predictions = loaded_model.predict(input_image)
    class_index = tf.argmax(predictions[0])
    class_label = class_index.numpy()

    # Append the result to the results list
    result = [image_path, class_label]
    results.append(result)

# Define the path to save the CSV file
csv_file = "/content/results.csv"

# Save the results to the CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image Path", "Class Label"])  # Write the header
    writer.writerows(results)  # Write the data rows

print("Results saved to:", csv_file)
